In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
from proj1_helpers import *
from data_helpers import *
from cross_validation import *
from plots import *

In [3]:
#LOAD TRAINING DATA
DATA_TRAIN_PATH = 'data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [4]:
#CLEAN TRAINING DATA
bounds = [0.0, 0.5]
tx_clean, ind_remov = treat_undefined_values(bounds, tX)
#tx_clean = remove_undefined_values (tX)
ind_remov

[4, 5, 6, 12, 26, 27, 28]

In [5]:
#MODEL BUILDING
tx, mean, std = standardize(tx_clean,0)

y = classify(y)
num_samples = len(y)
num_features = tx.shape[1]
y = y.reshape(num_samples,1)


In [6]:
num_samples, num_features
tx.shape

(250000, 23)

In [7]:
from compute_gradient import *
from cost import *
from implementations import *

In [8]:
#Initial weight


#Maximum iterations through the whole set
max_iter = 1000

#0.001 nan
#0.0005 nan

#Step-size
gamma = 0.001
gammas = np.logspace(-6,-3, 7)
lambdas = np.logspace (-5,0,7)
degrees = 7

#Regularization factor
# lambda_ = 0.01 142000 loss 10 000 iters
#lambda_ = 0.001 141000 loss 10 000 iters
lambda_ = 1e-5
#loss_min = np.inf
tx.shape,y.shape,y

((250000, 23), (250000, 1), array([[1.],
        [0.],
        [0.],
        ...,
        [1.],
        [0.],
        [0.]]))

In [9]:
loss_val_tot = []
loss_train_tot = []
k_fold = 2
loss_min = np.inf
#CHOOSE DEGREE FOR BUILD POLY
seed = 1
for ind, degree in enumerate(range(1,degrees+1)):
    tx_augmented = build_poly_all_features(tx,degree)
    y,tx_augmented = build_model_data(tx_augmented,y)
    print(tx_augmented.shape)
    loss_training = 0
    loss_validation = 0
    for i,k in enumerate(range(k_fold)):
        initial_w = np.zeros((tx_augmented.shape[1],1))
        tx_train, y_train, tx_val, y_val = cross_validation(y, tx_augmented, k, k_fold, seed)
        w, loss_train_k = reg_logistic_regression(y_train, tx_train, lambda_, initial_w, max_iter, gamma)
        loss_val_k = reg_logistic_loss (y_val,tx_val,w,lambda_)/len(y_val)
        loss_train_k /= len(y_train)
        loss_training += loss_train_k
        loss_validation += loss_val_k
    
        
    loss_training /= k_fold
    loss_validation /= k_fold
    print(loss_training)
    print(loss_validation)
    if (loss_validation < loss_min):
        weights_star = w
        degree_star = degree
        loss_min = loss_validation
        
    loss_val_tot.append(loss_validation)
    loss_train_tot.append(loss_training)



(250000, 24)
0.6171831137250321
0.6171516920549291
(250000, 47)


/Users/Sofia/Desktop/ML_Project1/cost.py:20: RuntimeWarning: divide by zero encountered in log
  loss = y.T.dot(np.log(pred)) + (1 - y).T.dot(np.log(1 - pred))


nan
inf
(250000, 70)
nan
nan
(250000, 93)
nan
nan
(250000, 116)
nan
nan
(250000, 139)


KeyboardInterrupt: 

In [ ]:
from plots import *
plot_cross_validation(range(1,1+degrees), "Degree", degree_star, loss_train_tot, loss_val_tot, loss_min)
print(" Best value of degree = {g} \n Loss = {l} \n Weights = {we}".format(
    g=degree_star, l=loss_min, we = weights_star))
weights_star.shape

In [179]:
#PREDICT TRAINING ACCURACY
tx_augmented = build_poly_all_features(tx,2)
y,tx_augmented = build_model_data(tx_augmented,y)
y_pred_training = predict_labels(weights_star, tx_augmented,'logistic')
count = 0
for i in range(num_samples):
    if (y_pred_training[i] == -1 and y[i] == 0):
        count += 1
    elif (y_pred_training[i] == y[i]):
        count += 1
accuracy = (count *100) / num_samples

accuracy

65.7348

In [198]:
#LOAD TEST SET
DATA_TEST_PATH = 'data/test.csv'
ytest, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [203]:
#CLEAN AND STANDARDIZE TEST SET
tX_test_clean = np.delete(tX_test, ind_remov, axis=1)
tX_test_clean.shape


(568238, 23)

In [206]:
tx_test_clean = remove_undefined_values (tX_test_clean)
tx_test_clean.shape

(568238, 23)

In [207]:
tx_test_clean, _, _ = standardize(tx_test_clean,0)


In [210]:
tx_test_augmented = build_poly_all_features(tx_test_clean,2)

(568238, 23)

In [213]:
tx_test_augmented.shape

(568238, 46)

In [214]:
#BUILD TEST MODEL
ytest, tx_test = build_model_data(tx_test_augmented,ytest)

In [215]:
#PREDICT LABELS
OUTPUT_PATH = 'data/submission_reg_logistic-degre2.csv'
y_pred = predict_labels(weights_star, tx_test,'logistic')
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)